In [1]:
from  __future__ import absolute_import
from __future__ import print_function
from keras.layers import *
from keras.models import Model, load_model
from keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from keras import backend as K
import tensorflow_hub as hub
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import logging
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_hub as hub
from PIL import Image
import pickle
import shutil
import matplotlib.pyplot as plt
import os
import random
import tensorflow as tf
from pathlib import Path
from keras import applications
from keras import layers
from keras import losses
from keras import optimizers
from keras import metrics
from keras import Model
from keras.applications import resnet
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import tensorflow_addons as tfa
from keras.regularizers import l2
from keras.activations import *
from DistanceLayer import DistanceLayer
from SiameseModel import SiameseModel
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

print(tf.__version__)


batch_size = 20



2.8.0


In [2]:
features = {}
file = "resnet"
# load the features dictionary from the file
with open('features_' + file + '.pickle', 'rb') as handle:
    features = pickle.load(handle)

feature_dim = features["02461"].shape[0]

print(features["02461"])
print(type(features["02461"]))
print(features["02461"][0])
print(features["02461"].shape)
print(len(features))

[3.2279101e-01 1.6950681e-03 2.0503240e+00 ... 1.2365297e+00 1.3910511e+00
 9.2304796e-01]
<class 'numpy.ndarray'>
0.322791
(2048,)
10000


# Model

a lot of the code comes from https://keras.io/examples/vision/siamese_network/
and https://github.com/akarshzingade/image-similarity-deep-ranking/blob/master/deepRanking.py

In [3]:
embedding = tf.keras.Sequential([
    Input(shape=(feature_dim,)),
    # BatchNormalization(),
    Dense(4096, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.001)),
    # BatchNormalization(),
    Dropout(0.6),
    
    Dense(4096, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.001)),
    # BatchNormalization(),
    Dropout(0.6),
    
    Lambda(lambda  x: K.l2_normalize(x,axis=1))
])

anchor_input = layers.Input(name="anchor", shape=(feature_dim))
positive_input = layers.Input(name="positive", shape=(feature_dim))
negative_input = layers.Input(name="negative", shape=(feature_dim))

distances = DistanceLayer()(
    embedding(anchor_input),
    embedding(positive_input),
    embedding(negative_input),
)

model = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

# model.summary()

In [4]:
#convert the features dictionary to a tensorflow hash table
keys = tf.constant([key for key in features.keys()])
values = tf.constant([val for val in features.values()])

table = tf.lookup.experimental.DenseHashTable(
    value_dtype=tf.float32,
    key_dtype=tf.string,
    empty_key="empty_key",
    deleted_key="deleted_key",
    default_value=[-1]*feature_dim,
    )

table.insert(keys, values)

In [5]:
train_triplets = np.loadtxt('./train_triplets.txt', dtype=str,delimiter = ' ')
val_triplets = np.loadtxt('./val_triplets_split.txt', dtype=str,delimiter = ' ')
train_dataset = tf.data.Dataset.from_tensor_slices(train_triplets)
val_dataset = tf.data.Dataset.from_tensor_slices(val_triplets)
count = np.array(train_triplets).shape[0]
print(count)
print(np.array(val_triplets).shape[0])


@tf.function
def load_image(inputs):
    filenames = inputs
    anchor = table[filenames[0]]
    positive = table[filenames[1]]
    negative = table[filenames[2]]
    output = (anchor,positive,negative)
    
    return output

train_dataset = train_dataset.map(load_image).cache().shuffle(buffer_size=1000).batch(batch_size).prefetch(20)
val_dataset = val_dataset.map(load_image).cache().batch(100).prefetch(20)

59520
7471


In [6]:
model = SiameseModel(model,margin=0.5,scale=64)
model.compile(optimizer=tf.keras.optimizers.Adam(0.000001))


In [15]:
train_steps_per_epoch = int(count/batch_size)
# checkpoint_filepath = './checkpoints/checkpoint-{epoch}/'
#save checkpoint after every epoch
# callback = ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     # save_weights_only=True,
# )
# callback = EarlyStopping(monitor='val_accuracy', patience=4,restore_best_weights=True)


model.fit(train_dataset, epochs=1,steps_per_epoch=train_steps_per_epoch,batch_size=batch_size,validation_data=val_dataset)#,callbacks=[callback]




2976/2976 [==============================] - 49s 16ms/step - loss: 0.2428 - val_accuracy: 0.8196


# this code was used to initialize the distances file

In [16]:
# #predict distances and save them to a file
# set_file = "test"
# dataset = np.loadtxt('./'+set_file+'_triplets.txt', dtype=str,delimiter = ' ')
# print(dataset.shape)
# dataset = tf.data.Dataset.from_tensor_slices(dataset)
# dataset = dataset.map(load_image).batch(100)
# data = []
# for triplet in iter(dataset):
#     anchor = triplet[0]
#     positive = triplet[1]
#     negative = triplet[2]
#     distances = model.predict([anchor,positive,negative])
#     # print(np.array(distances[0]).shape)
#     # print(np.array(distances[1]).shape)
#     s = np.array(distances).shape[1]
#     for i in range(s):
#         data.append([distances[0][i],distances[1][i]])



# df = pd.DataFrame(data=data,columns=['anchor_left_distance_'+file,'anchor_right_distance_'+file])
# df.to_csv(set_file+'_distances.csv')




# append other distances

In [17]:
#predict distances and save them to a file
set_file = "test6"
dataset = np.loadtxt('./test_triplets.txt', dtype=str,delimiter = ' ')
print(dataset.shape)
dataset = tf.data.Dataset.from_tensor_slices(dataset)
dataset = dataset.map(load_image).batch(100)
left = []
right = []
for triplet in iter(dataset):
    anchor = triplet[0]
    positive = triplet[1]
    negative = triplet[2]
    distances = model.predict([anchor,positive,negative])
    left_distances = distances[0]
    right_distances = distances[1]
    s = np.array(left_distances).shape[0]
    # print(s)
    for i in range(s):
        left.append(distances[0][i])
        right.append(distances[1][i])



df = pd.read_csv(set_file+"_distances.csv",index_col=0)
df['anchor_left_distance_'+file] = left
df['anchor_right_distance_'+file] = right
df.to_csv(set_file+'_distances.csv')



(59544, 3)


# using clip directly (sucks)

In [18]:
# #predict distances and save them to a file
# set_file = "val"
# dataset = np.loadtxt('./'+set_file+'_triplets_split_shuffled.txt', dtype=str,delimiter = ' ')
# print(dataset.shape)
# left = []
# right = []
# for triplet in iter(dataset):
#     anchor = triplet[0]
#     positive = triplet[1]
#     negative = triplet[2]
#     # print(np.sum((features[anchor])**2))
#     left.append(np.sum((features[anchor]-features[positive])**2))
#     right.append(np.sum((features[anchor]-features[negative])**2))


# df = pd.read_csv(set_file+"_distances.csv",index_col=0)
# df['anchor_left_distance_clip2'] = left
# df['anchor_right_distance_clip2'] = right
# df.to_csv(set_file+'_distances.csv')

